<a href="https://colab.research.google.com/github/Kobayashi139/food-supply/blob/main/%E3%83%AD%E3%83%BC%E3%83%AA%E3%83%B3%E3%82%B0%E3%83%9B%E3%83%A9%E3%82%A4%E3%82%BA%E3%83%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ライブラリ

In [ ]:
# =================================================================
# モジュール1: 環境設定とライブラリのインポート
# =================================================================
!pip install -q pyomo plotly
!apt-get install -y -qq glpk-utils

import pyomo.environ as pyo # 数理最適化
import pandas as pd # テーブル操作
import numpy as np # 数値計算、配列など
import random, time # 乱数、速度確認
import plotly.express as px # グラフ描画（簡単
import plotly.graph_objects as go # グラフ描画（本格
from plotly.subplots import make_subplots # グラフを並べる

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

In [ ]:
try:
    # 乱数シードを固定し、毎回同じ乱数が発生するようにする
    np.random.seed(42)
    solver = pyo.SolverFactory('glpk')
except pyo.common.errors.ApplicationError: # error時
    print("GLPK solver not found. Please ensure it is installed and in your system's PATH.")
    solver = None  # null

## パラメータ

In [ ]:
# =================================================================
# モジュール2: パラメータとシナリオの定義
# =================================================================

# --- 製品タイプの集合 ---
# I = {f: 生鮮品（檸檬）, a: 加工品A（檸檬タルト）, b: 加工品B（檸檬シロップ）}
products = ['f', 'a', 'b']
processed_products = ['a', 'b']

# --- 単位統一のための基本データ設定 ---
# 製品1個あたりの基本情報
# タルト('a')
WEIGHT_TART_g = 350      # 容量 (g/個)
LEMON_PER_TART_ml = 50    # 檸檬含量 (ml/個)
LIFESPAN_TART_days = 120   # 保存期間 (日)
SALES_TART = 3800 # 値段 (円/個)
# シロップ('b')
WEIGHT_SYRUP_g = 400     # 容量 (g/個)
LEMON_PER_SYRUP_ml = 200  # 檸檬含量 (ml/個)
LIFESPAN_SYRUP_days = 365 # 保存期間 (日)
SALES_SYRUP = 2200 # 値段 (円/個)
# 生鮮品('f') - 檸檬
LIFESPAN_FRESH_days = 60  # 保存期間 (日、仮定)
SALES_FRESH = 500 # 値段 (円/kg)

# 1個あたりの重量 (kg/個)
WEIGHT_PER_PIECE = {
    'a': WEIGHT_TART_g / 1000,
    'b': WEIGHT_SYRUP_g / 1000
}
# 製品1個の製造に必要な檸檬の重量 (kg/個) - 檸檬の密度を約1g/mlと仮定
LEMON_KG_PER_PIECE = {
    'a': LEMON_PER_TART_ml / 1000,
    'b': LEMON_PER_SYRUP_ml / 1000
}
print(WEIGHT_PER_PIECE)
print(LEMON_KG_PER_PIECE)

# 電気代・ガス代 (円/月)
COST_FREEZER_monthly = 389.17 # 冷凍庫
COST_OVEN_monthly = 155       # オーブン
COST_GAS_monthly = 1048.5     # ガスコンロ

# 梱包費など (円/個)
PACKING_COST_PER_PIECE = {
    'a': 63,
    'b': 230
}

# --- コスト、劣化率、効率などのパラメータ ---

# 1. 保管コスト h (円/kg・月)
# 月額コストを、その保管場所で保管可能な最大重量(kg)で割って算出
h_a = COST_FREEZER_monthly / 80  # 冷凍庫100Lに80kg保管可能と仮定
h_b = 1                          # 常温保管コストは微小と仮定
h_f = 5                          # 冷蔵庫の月額コスト

# 2. 劣化率 gamma (%/月)
# 1ヶ月(30日)で劣化する割合。 (30日 / 保存期間) で計算、上限1.0(100%)
gamma_f = min(1.0, 30 / LIFESPAN_FRESH_days)
gamma_a = min(1.0, 30 / LIFESPAN_TART_days)
gamma_b = min(1.0, 30 / LIFESPAN_SYRUP_days)


# 3. 加工効率 e (kg-output / kg-input)
e_a = 0.8
e_b = 0.9

# 4. 加工コスト p (円/kg-input)
# 原料(檸檬)1kgを加工するためのコスト」
# (設備費＋梱包費)を、対応する「原料投入量(kg-input)」で割る

# タルト('a'): 月産60個と仮定
p_a_monthly_cost = COST_OVEN_monthly + (PACKING_COST_PER_PIECE['a'] * 80)
p_a_monthly_weight_kg = WEIGHT_PER_PIECE['a'] * 60 # 月間「生産量(output)」
#「投入量(input)」を効率e_aで逆算
p_a_monthly_input_kg = p_a_monthly_weight_kg / e_a
# コストを「投入量」で割る
p_a = p_a_monthly_cost / p_a_monthly_input_kg

# シロップ('b'): 月産30個と仮定
p_b_monthly_cost = COST_GAS_monthly + (PACKING_COST_PER_PIECE['b'] * 40)
p_b_monthly_weight_kg = WEIGHT_PER_PIECE['b'] * 30 # 月間「生産量(output)」
# 対応する「投入量(input)」を効率e_bで逆算
p_b_monthly_input_kg = p_b_monthly_weight_kg / e_b
# コストを「投入量」で割る
p_b = p_b_monthly_cost / p_b_monthly_input_kg

# 販売価格（終端価値のため）
sales_prices = {'f': SALES_FRESH, 'a': SALES_TART, 'b': SALES_SYRUP}

# h_i: 保管コスト, p_i: 加工コスト, γ_i: 劣化率, φ_i: 廃棄コスト, e_i: 加工効率, o: 1単位あたりの機会損失コスト
params = {
    'h': {'f': h_f, 'a': h_a, 'b': h_b},
    'p': {'f': 0, 'a': p_a, 'b': p_b},
    'gamma': {'f': gamma_f, 'a': gamma_a, 'b': gamma_b},
    'e': {'f': 1, 'a': e_a, 'b': e_b},
    'phi': {'f': 1000, 'a': 1000, 'b': 1500}, # 処理コスト重視(円/kg)
    'o': {'f': 500, 'a': 2000, 'b': 1500}   # 粗利寄りの計上(円/kg)
}

print(params['p'])

# --- 能力に関するパラメータ (単位: kg) ---
capacities = {
    # U: 在庫上限 (kg)
    'U': {'f': 40, 'a': 80, 'b': 120},
    # u: 月間の最大生産量 (kg/月)
    'u': {
        'a': 60 * WEIGHT_PER_PIECE['a'], # 月最大60個生産と仮定
        'b': 30 * WEIGHT_PER_PIECE['b']   # 月最大30個生産と仮定
    }
}

# --- 供給と初期在庫 (単位: kg) ---
C_supply = 25 # 月間の生鮮品(檸檬)の供給量 (kg)
initial_inventory_post = {'f': 5, 'a': 0, 'b': 0} # 加工後在庫 (kg)
initial_inventory_pre = {'a': 0, 'b': 0}          # 加工前在庫(原料檸檬) (kg)

# --- 需要パラメータ ---
demand_params = {
    'mean': {
        'f': 5, # 生の檸檬の月間需要 (kg)
        'a': 30 * WEIGHT_PER_PIECE['a'], # タルト: 月需要(個
        'b': 20 * WEIGHT_PER_PIECE['b']  # シロップ: 月需要(個
    },
    'std': {
        'f': 2,
        'a': 15 * WEIGHT_PER_PIECE['a'],
        'b': 10 * WEIGHT_PER_PIECE['b']
    }
}

num_scenarios_per_period = 20 # 生成するシナリオの数

params['waste_limit'] = 0.5 * C_supply

# --- 環境負荷係数（ポイント/kg = おおむね kgCO2e/kg に対応） ---

waste_emission_factors = {
    'low': { 'f': 0.90, 'a': 0.90, 'b': 0.90 },
    'middle': {
        # [出典] 農業LCI(3.64t-CO2/百万円) (1.09)
        'f': 1.00,

        # 原料(1.09) + 電気加工(0.26) + 複合焼却(0.16) = 1.51
        'a': 1.51,

        # 原料(1.09) + ガス加工(0.12) + 瓶ロス/処理(0.33) = 1.54
        'b': 1.54
    },
    'high': { 'f': 4.95, 'a': 5.65, 'b': 5.50 }
}

process_emission_factors = {
    'low': { 'a': 0.05, 'b': 0.05 },
    'middle': {
        # [出典] 中国電力R5 (0.511 kg/kWh) * 0.5 kWh/kg
        'a': 0.26,
        # [出典] 広島ガスR5 (2.05 kg/m3) * 0.06 m3/kg
        'b': 0.12
    },
    'high': { 'a': 0.50, 'b': 0.30 }
}


{'a': 0.35, 'b': 0.4}
{'a': 0.05, 'b': 0.2}
{'f': 0, 'a': 197.9047619047619, 'b': 768.6375}


## パレート

In [ ]:
# =================================================================
# モジュール3: 多目的最適化モデルを解く関数
# =================================================================
import shutil # ソルバーのパス確認用

# --- モジュール3-1 ---
def solve_multi_objective_model(initial_inv_post, initial_inv_pre, supply, scenarios_list, params, caps, env_weight, normalization_limits=None, sales_prices=None):
    model = pyo.ConcreteModel("Multi_Objective_Rolling_Model") # 最適化問題の箱を作る

    # --- 集合の定義( pyo.Set ) --- パラメータの値を入れている
    T_horizon = len(scenarios_list) # 何期分か
    model.T = pyo.Set(initialize=range(T_horizon)) # 期間の集合 t=0, 1, ..., T-1 initialize=初期化
    model.I = pyo.Set(initialize=params['h'].keys())
    model.I_processed = pyo.Set(initialize=[i for i in params['h'].keys() if i != 'f'])

    scenario_keys = scenarios_list[0].keys() # 1期ごとの全シナリオを入れる
    model.S = pyo.Set(initialize=scenario_keys)

    # --- 決定変数( pyo.Var ) --- GLPKに決めさせる変数
    # NonNegativeReals= 非負制約
    model.X = pyo.Var(model.T, model.I, domain=pyo.NonNegativeReals) # 原料配分量
    model.P_actual = pyo.Var(model.T, model.I_processed, domain=pyo.NonNegativeReals) # 加工投入量
    model.I_end_post = pyo.Var(model.T, model.S, model.I, domain=pyo.NonNegativeReals) # 期末在庫(加工後)
    model.I_end_pre = pyo.Var(model.T, model.S, model.I_processed, domain=pyo.NonNegativeReals) # 期末在庫(加工前)
    model.S_sold = pyo.Var(model.T, model.S, model.I, domain=pyo.NonNegativeReals) # 販売量
    model.W_waste_post = pyo.Var(model.T, model.S, model.I, domain=pyo.NonNegativeReals) # 廃棄量(加工後)
    model.W_waste_pre = pyo.Var(model.T, model.S, model.I_processed, domain=pyo.NonNegativeReals) # 廃棄量(加工前)
    model.W_waste_proc = pyo.Var(model.T, model.I_processed, domain=pyo.NonNegativeReals) # 加工廃棄
    model.UnmetDemand = pyo.Var(model.T, model.S, model.I, domain=pyo.NonNegativeReals) # 未充足需要
    model.waste_emission_factors = waste_emission_factors # 廃棄環境負荷
    model.process_emission_factors = process_emission_factors  # 加工環境負荷

    # --- 目的関数要素の定義 ---
    # 経済コスト
    def calc_economic_cost(m): # m = modelを受け取る 以降は「m.」で呼び出し
        total_cost = 0

        # 終端在庫の評価係数（販売価格の何割の価値とみなすか）
        TERMINAL_VALUE_RATIO = 0.7

        for t in m.T: # 最適化6カ月分
            scenarios = scenarios_list[t]

            # 保管コスト (for i in m.I) = 製品集合から１種ずつ取り出す
            term_h_post = sum(scenarios[s]['prob'] * sum(params['h'][i] * m.I_end_post[t, s, i] for i in m.I) for s in m.S)
            term_h_pre = sum(scenarios[s]['prob'] * sum(params['h']['f'] * m.I_end_pre[t, s, i] for i in m.I_processed) for s in m.S)

            # 加工コスト (生産活動そのもののコスト)
            term_p = sum(params['p'][i] * m.P_actual[t, i] for i in m.I_processed)

            # 廃棄コスト (劣化廃棄 + 加工廃棄)
            # 劣化廃棄 (確率的)
            term_w_deg = sum(scenarios[s]['prob'] * (
                sum(params['phi'][i] * m.W_waste_post[t, s, i] for i in m.I) +
                sum(params['phi']['f'] * m.W_waste_pre[t, s, i] for i in m.I_processed)
            ) for s in m.S)

            # 加工廃棄 (確定的: P_actualに依存) -> 処理コスト phi を適用
            term_w_proc = sum(params['phi'][i] * m.W_waste_proc[t, i] for i in m.I_processed)

            # 機会損失
            term_opp = sum(scenarios[s]['prob'] * sum(params['o'][i] * m.UnmetDemand[t, s, i] for i in m.I) for s in m.S)

            total_cost += (term_h_post + term_h_pre + term_p + term_w_deg + term_w_proc + term_opp)

            # 計画期間の最終月(T_horizon - 1)の期末在庫に対して、価値を認めてコストから引く
            # sales_prices is not None = 変数が存在していないならば
            if t == T_horizon - 1 and sales_prices is not None:
                # 期待在庫量に対して評価を行う
                expected_inv_post = sum(scenarios[s]['prob'] * sum(m.I_end_post[t, s, i] * sales_prices[i] for i in m.I) for s in m.S)
                expected_inv_pre = sum(scenarios[s]['prob'] * sum(m.I_end_pre[t, s, i] * sales_prices['f'] for i in m.I_processed) for s in m.S) # 原料在庫は生鮮価格で評価

                terminal_value = (expected_inv_post + expected_inv_pre) * TERMINAL_VALUE_RATIO # 在庫の7割を残存コストとする
                total_cost -= terminal_value # 総コストから有益在庫分コストを引く

        return total_cost

    # 環境負荷
    def calc_env_score(m):
        total_score = 0

        for t in m.T:
            scenarios = scenarios_list[t]

            # 廃棄負荷 (Middle)
            # 劣化廃棄
            term_env_w_deg = sum(scenarios[s]['prob'] * (
                sum(m.waste_emission_factors['middle'][p] * m.W_waste_post[t, s, p] for p in m.I) +
                sum(m.waste_emission_factors['middle']['f'] * m.W_waste_pre[t, s, i] for i in m.I_processed)
            ) for s in m.S)

            # 加工廃棄 -> 廃棄排出係数を適用
            term_env_w_proc = sum(m.waste_emission_factors['middle'][p] * m.W_waste_proc[t, p] for p in m.I_processed)

            # 加工負荷 (エネルギー使用など)
            term_env_p = sum(m.process_emission_factors['middle'][p] * m.P_actual[t, p] for p in m.I_processed)

            total_score += (term_env_w_deg + term_env_w_proc + term_env_p)
        return total_score

    # Expression = 数式
    model.economic_cost = pyo.Expression(rule=calc_economic_cost) # 経済コストの数式を登録する
    model.environmental_score = pyo.Expression(rule=calc_env_score) # 環境負荷の数式を登録する

    # 目的関数の正規化
    def objective_rule(m):
        cost_term = m.economic_cost
        env_term = m.environmental_score
        if normalization_limits:
            c_min, c_max, e_min, e_max = normalization_limits
            denom_c = c_max - c_min if (c_max - c_min) > 1e-5 else 1.0 # 差が小さい場合は分母を1.0にする
            denom_e = e_max - e_min if (e_max - e_min) > 1e-5 else 1.0
            norm_cost = (cost_term - c_min) / denom_c
            norm_env  = (env_term - e_min) / denom_e
            return (1.0 - env_weight) * norm_cost + env_weight * norm_env
        else:
            return (1.0 - env_weight) * cost_term + env_weight * env_term

    # 目的関数の数式を登録する
    model.total_objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize) # Objective = 目的関数のリスト

    # --- 制約条件 ---
    model.constraints = pyo.ConstraintList() # 制約のリストを表す制約コンポーネント

    for t in model.T:
        scenarios = scenarios_list[t]

        # 1. 総供給量制約（供給はすべて割り振る）
        # .add = 制約リストに追加する
        model.constraints.add(sum(model.X[t, i] for i in model.I) == supply)

        # 2. 加工能力と原料利用の制約
        for i in model.I_processed:
            model.constraints.add(model.P_actual[t, i] * params['e'][i] <= caps['u'][i])
            model.constraints.add(model.W_waste_proc[t, i] == model.P_actual[t, i] * (1 - params['e'][i]))

            if t == 0:
                prev_inv = initial_inv_pre.get(i, 0)
            else:
                prev_inv = sum(scenarios_list[t-1][s]['prob'] * model.I_end_pre[t-1, s, i] for s in model.S)

            model.constraints.add(model.P_actual[t, i] <= prev_inv * (1 - params['gamma']['f']) + model.X[t, i])

        # 3. 各シナリオにおけるバランス制約
        for s in model.S:
            # --- 生鮮品 (f) ---
            if t == 0:
                inv_post_f = initial_inv_post['f']
            else:
                inv_post_f = model.I_end_post[t-1, s, 'f']
                inv_post_f = sum(scenarios_list[t-1][ss]['prob'] * model.I_end_post[t-1, ss, 'f'] for ss in model.S)

            model.constraints.add(
                inv_post_f * (1 - params['gamma']['f']) + model.X[t, 'f']
                == model.S_sold[t, s, 'f'] + model.I_end_post[t, s, 'f']
            )

            # --- 加工品 (a, b) ---
            for i in model.I_processed:
                if t == 0:
                    inv_post_proc = initial_inv_post.get(i, 0)
                    inv_pre_proc = initial_inv_pre.get(i, 0)
                else:
                    inv_post_proc = sum(scenarios_list[t-1][ss]['prob'] * model.I_end_post[t-1, ss, i] for ss in model.S)
                    inv_pre_proc = sum(scenarios_list[t-1][ss]['prob'] * model.I_end_pre[t-1, ss, i] for ss in model.S)

                model.constraints.add(
                    inv_post_proc * (1 - params['gamma'][i]) + model.P_actual[t, i] * params['e'][i]
                    == model.S_sold[t, s, i] + model.I_end_post[t, s, i]
                )

                model.constraints.add(
                    (inv_pre_proc * (1 - params['gamma']['f']) + model.X[t, i]) - model.P_actual[t, i]
                    == model.I_end_pre[t, s, i]
                )
                model.constraints.add(model.W_waste_pre[t, s, i] == inv_pre_proc * params['gamma']['f'])

            # --- 需要・廃棄・上限 ---
            for i in model.I:
                model.constraints.add(model.S_sold[t, s, i] <= scenarios[s]['d'][i])
                model.constraints.add(model.S_sold[t, s, i] + model.UnmetDemand[t, s, i] == scenarios[s]['d'][i])

                if i == 'f': start_inv = inv_post_f
                else:
                    if t == 0: start_inv = initial_inv_post.get(i, 0)
                    else: start_inv = sum(scenarios_list[t-1][ss]['prob'] * model.I_end_post[t-1, ss, i] for ss in model.S)

                model.constraints.add(model.W_waste_post[t, s, i] == start_inv * params['gamma'][i])
                model.constraints.add(model.I_end_post[t, s, i] <= caps['U'][i])


    # --- ソルバーの実行 ---
    # グローバル変数のsolverではなく、関数内で確実にglpkを見つけて実行する
    glpsol_path = shutil.which("glpsol")
    if glpsol_path:
        opt = pyo.SolverFactory('glpk', executable=glpsol_path) # Pyomoが書いた数式モデルを読み、最適化を実行する
    else:
        # パスが見つからない場合はデフォルト動作
        opt = pyo.SolverFactory('glpk')

    try:
        results = opt.solve(model, tee=False) # tee=Trueでソルバーログが出る
    except Exception as e:
        print(f"Solver Error: {e}")
        print("ヒント: ランタイムがリセットされた可能性 !apt-get install glpk-utils を再実行してください。")
        return None, None

    # results = メタ情報、.status = 動いたか(ok=OK)、.termination_conditio = 解が見つかったか(optimal=最適解見つかった)
    if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
        decisions = {
            # 製品iごとに、今期(t=0)の原料配分量を数値として取り出す
            'X': {i: pyo.value(model.X[0, i]) for i in model.I},
            # 加工投入量
            'P_actual': {i: pyo.value(model.P_actual[0, i]) for i in model.I_processed}
        }
        # decisions= 次期の在庫更新用、model= Pyomoモデル丸ごと（評価用）
        return decisions, model
    else:
        return None, None


# --- モジュール3-2 : 正規化の基準値を探す ---
def calculate_normalization_bounds(initial_inv_post, initial_inv_pre, supply, scenarios_list, params, caps, weight_grid=None):
    print(f"正規化のための基準値を計算中 (Horizon: {len(scenarios_list)} months)...")

    if weight_grid is None:
        # 正規化用の粗い探索グリッド（標準的）
        # 重みの候補リスト、np.linspace(開始,終了,間を何等分するか)
        weight_grid = np.linspace(0.0, 1.0, 11)

    print(f"正規化のための基準値を計算中 "
          f"(Horizon: {len(scenarios_list)} months, "
          f"Grid size: {len(weight_grid)})...")

    cost_values = []
    env_values = []

    # 正規化なしで最適化を回す、ローリングホライズンなし
    # 6カ月分を一括でλ(0～1)変化させる
    for w in weight_grid:
        _, model = solve_multi_objective_model( # モジュール3-1（最適化）
                                                # _, = １個目の返り値(decisions)を使用しない
            initial_inv_post, # 初期在庫
            initial_inv_pre, # 加工前初期在庫
            supply, # 供給量
            scenarios_list, # 確率シナリオ
            params, # コストパラメータ
            caps, # 制約
            env_weight=w, # 重み
            normalization_limits=None  # ← 正規化しない
        )

        if model is None:
            raise RuntimeError(f"ソルバーエラー (env_weight={w})")

        cost = pyo.value(model.economic_cost)
        env  = pyo.value(model.environmental_score)

        cost_values.append(cost)
        env_values.append(env)

    # 現実的な限界値を記録
    c_min = min(cost_values)
    c_max = max(cost_values)
    e_min = min(env_values)
    e_max = max(env_values)

    print(f"  Cost Range (Total for Horizon): {c_min:,.0f} ~ {c_max:,.0f}")
    print(f"  Env  Range (Total for Horizon): {e_min:,.3f} ~ {e_max:,.3f}")

    return c_min, c_max, e_min, e_max

# =================================================================
# モジュール4: シミュレーション (実績値詳細計算・ローリングホライズン)
# =================================================================
def run_two_stage_simulation(simulation_periods, env_weight, initial_inventory_post, initial_inventory_pre, master_scenarios, normalization_limits=None, verbose=True):
    HORIZON_LENGTH = 6 # 計画期間 (当期+未来5ヶ月)

    if verbose:
        print("\n" + "="*80)
        print(f"詳細シミュレーション実行中 (ローリングホライズン T={HORIZON_LENGTH}, λ={env_weight:.2f})")
        print("="*80)

    # 初期在庫の設定
    inv_post = initial_inventory_post.copy()
    inv_pre = initial_inventory_pre.copy()
    full_history = []

    for t in range(1, simulation_periods + 1):
        # 1. ローリングホライズン用のシナリオ切り出し (当期 t から Tヶ月分)
        current_scenarios_list = []
        for h in range(HORIZON_LENGTH):
            target_idx = (t - 1) + h
            # シナリオが足りない場合は最終期間のものをコピーして埋める（エラー対策）
            if target_idx < len(master_scenarios):
                current_scenarios_list.append(master_scenarios[target_idx])
            else:
                current_scenarios_list.append(master_scenarios[-1])

        # 2. 最適化実行 (6ヶ月先まで考慮して決定)
        decisions, solved_model = solve_multi_objective_model( # モジュール3-1（最適化）
            inv_post, inv_pre, C_supply, current_scenarios_list,
            params, capacities, env_weight, normalization_limits
        )

        if not decisions:
            print(f"❌ {t}期で最適解なし。中断。")
            break
        # 3. 実績値の計算
        scenarios = current_scenarios_list[0] # 当期のシナリオ

        # 決定変数
        # i:ループ、value:数値変換、solved_model:計算済みモデル、products:全製品タイプ
        val_X = {i: pyo.value(solved_model.X[0, i]) for i in products}
        val_P = {i: pyo.value(solved_model.P_actual[0, i]) for i in processed_products}

        # 在庫量の期待値
        val_Inv_post = {i: sum(scenarios[s]['prob'] * pyo.value(solved_model.I_end_post[0, s, i]) for s in scenarios) for i in products}
        val_Inv_pre  = {i: sum(scenarios[s]['prob'] * pyo.value(solved_model.I_end_pre[0, s, i]) for s in scenarios) for i in processed_products}

        # 劣化廃棄
        val_W_post = {i: sum(scenarios[s]['prob'] * pyo.value(solved_model.W_waste_post[0, s, i]) for s in scenarios) for i in products}
        val_W_pre  = {i: sum(scenarios[s]['prob'] * pyo.value(solved_model.W_waste_pre[0, s, i]) for s in scenarios) for i in processed_products}

        # 加工廃棄
        val_W_proc = {i: pyo.value(solved_model.W_waste_proc[0, i]) for i in processed_products}
        val_Unmet = {i: sum(scenarios[s]['prob'] * pyo.value(solved_model.UnmetDemand[0, s, i]) for s in scenarios) for i in products}

        # コスト計算
        cost_holding = sum(params['h'][i] * val_Inv_post[i] for i in products) + sum(params['h']['f'] * val_Inv_pre[i] for i in processed_products)
        cost_processing = sum(params['p'][i] * val_P[i] for i in processed_products)

        # 廃棄コスト (劣化 + 加工)
        cost_waste_deg = sum(params['phi'][i] * val_W_post[i] for i in products) + sum(params['phi']['f'] * val_W_pre[i] for i in processed_products)
        cost_waste_proc = sum(params['phi'][i] * val_W_proc[i] for i in processed_products)
        cost_waste = cost_waste_deg + cost_waste_proc

        cost_opportunity = sum(params['o'][i] * val_Unmet[i] for i in products)
        cost_total = cost_holding + cost_processing + cost_waste + cost_opportunity

        # 環境負荷計算 (Middleのみ)
        # 廃棄由来 (劣化 + 加工)
        env_waste_deg = sum(waste_emission_factors['middle'][i] * val_W_post[i] for i in products) + sum(waste_emission_factors['middle']['f'] * val_W_pre[i] for i in processed_products)
        env_waste_proc = sum(waste_emission_factors['middle'][i] * val_W_proc[i] for i in processed_products)
        env_waste = env_waste_deg + env_waste_proc

        env_process = sum(process_emission_factors['middle'][i] * val_P[i] for i in processed_products)
        env_total = env_waste + env_process

        # データ記録
        period_data = {
            'Period': t,
            'Total Cost': cost_total,
            'Holding Cost': cost_holding, 'Processing Cost': cost_processing,
            'Waste Cost': cost_waste, 'Opportunity Cost': cost_opportunity,
            'Total Env Score': env_total, 'Waste Env': env_waste, 'Process Env': env_process,
            'Inv_post_f': val_Inv_post['f'], 'Inv_post_a': val_Inv_post['a'], 'Inv_post_b': val_Inv_post['b'],
            'Inv_pre_a': val_Inv_pre['a'], 'Inv_pre_b': val_Inv_pre['b'],
            'X_f': val_X['f'], 'X_a': val_X['a'], 'X_b': val_X['b'],
            'P_a': val_P['a'], 'P_b': val_P['b'],
            'W_post_f': val_W_post['f'], 'W_post_a': val_W_post['a'], 'W_post_b': val_W_post['b'],
            'W_pre_a': val_W_pre['a'], 'W_pre_b': val_W_pre['b'],
            'W_proc_a': val_W_proc['a'], 'W_proc_b': val_W_proc['b']
        }
        full_history.append(period_data) # データを配列に追加する

        # 4. 次期への在庫更新
        inv_post = {p: period_data[f'Inv_post_{p}'] for p in products}
        inv_pre = {p: period_data[f'Inv_pre_{p}'] for p in processed_products}

    # pd:Pandas、DataFrame:行列の表に整形する
    return pd.DataFrame(full_history)

# =================================================================
# モジュール5: パレートフロンティア分析
# =================================================================
def run_pareto_analysis(env_weight_list, simulation_periods, master_scenarios, normalization_limits=None, title="パレートフロンティア分析"):
    print("="*80 + f"\n{title}を開始...\n" + "="*80)
    results_list = []

    for weight in env_weight_list:
        # 各λでシミュレーションを実行
        # モジュール4（ローリングホライズン）を試行、λを変化させつつ12ヶ月分
        df_history = run_two_stage_simulation(simulation_periods, env_weight=weight,
                                                  initial_inventory_post=initial_inventory_post,
                                                  initial_inventory_pre=initial_inventory_pre,
                                                  master_scenarios=master_scenarios,
                                                  normalization_limits=normalization_limits,
                                                  verbose=False)
        if not df_history.empty:
            # シミュレーション結果(df_history)= 実績コストのため、合計するだけでよい
            total_cost = df_history['Total Cost'].sum()
            total_env = df_history['Total Env Score'].sum()

            results_list.append({
                'weight': weight,
                'total_cost': total_cost,
                'total_env_score': total_env,
                'total_objective': (1 - weight) * total_cost + weight * total_env
            })

    return pd.DataFrame(results_list)

# =================================================================
# モジュール6: 1期間の詳細分析と結果表示
# =================================================================
def analyze_and_print_period_details(model, scenarios, params):
    t = 0 # 最適化モデル内の t=0 (当期) の情報を表示

    # コスト計算
    holding_cost_post = sum(scenarios[s]['prob'] * sum(params['h'][i] * pyo.value(model.I_end_post[t, s, i]) for i in model.I) for s in model.S)
    holding_cost_pre = sum(scenarios[s]['prob'] * sum(params['h']['f'] * pyo.value(model.I_end_pre[t, s, i]) for i in model.I_processed) for s in model.S)
    processing_cost = sum(params['p'][i] * pyo.value(model.P_actual[t, i]) for i in model.I_processed)

    # 廃棄コスト (劣化)
    waste_cost_deg = sum(scenarios[s]['prob'] * (
        sum(params['phi'][i] * pyo.value(model.W_waste_post[t, s, i]) for i in model.I) +
        sum(params['phi']['f'] * pyo.value(model.W_waste_pre[t, s, i]) for i in model.I_processed)
    ) for s in model.S)

    # 廃棄コスト (加工)
    waste_cost_proc = sum(params['phi'][i] * pyo.value(model.W_waste_proc[t, i]) for i in model.I_processed)

    opportunity_loss_cost = sum(scenarios[s]['prob'] * sum(params['o'][i] * pyo.value(model.UnmetDemand[t, s, i]) for i in model.I) for s in model.S)
    total_cost_period = holding_cost_post + holding_cost_pre + processing_cost + waste_cost_deg + waste_cost_proc + opportunity_loss_cost

    # ロス量計算
    spoilage_f = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_post[t, s, 'f']) for s in model.S)
    spoilage_post_a = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_post[t, s, 'a']) for s in model.S)
    spoilage_pre_a = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_pre[t, s, 'a']) for s in model.S)
    processing_loss_a = pyo.value(model.W_waste_proc[t, 'a']) # 直接変数から取得

    spoilage_post_b = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_post[t, s, 'b']) for s in model.S)
    spoilage_pre_b = sum(scenarios[s]['prob'] * pyo.value(model.W_waste_pre[t, s, 'b']) for s in model.S)
    processing_loss_b = pyo.value(model.W_waste_proc[t, 'b']) # 直接変数から取得

    total_food_loss = spoilage_f + spoilage_post_a + spoilage_pre_a + spoilage_post_b + spoilage_pre_b + processing_loss_a + processing_loss_b

    # 環境負荷計算 (Middle)
    level = 'middle'
    waste_impact_deg = sum(scenarios[s]['prob'] * (
        sum(waste_emission_factors[level][i] * pyo.value(model.W_waste_post[t, s, i]) for i in model.I) +
        sum(waste_emission_factors[level]['f'] * pyo.value(model.W_waste_pre[t, s, i]) for i in model.I_processed)
    ) for s in model.S)
    waste_impact_proc = sum(waste_emission_factors[level][i] * pyo.value(model.W_waste_proc[t, i]) for i in model.I_processed)

    process_impact = sum(process_emission_factors[level][i] * pyo.value(model.P_actual[t, i]) for i in model.I_processed)
    total_impact = waste_impact_deg + waste_impact_proc + process_impact

    # 表示
    print(f"期待総コスト (当期分): {total_cost_period:,.3f} 円")
    print("\n--- 総コストの内訳 (当期分) ---")
    print(f"  期待保管コスト        : {(holding_cost_post + holding_cost_pre):,.3f}")
    print(f"  加工コスト            : {processing_cost:,.3f}")
    print(f"  期待廃棄コスト        : {(waste_cost_deg + waste_cost_proc):,.3f} (内 加工廃棄: {waste_cost_proc:,.3f})")
    print(f"  期待機会損失コスト    : {opportunity_loss_cost:,.3f}")
    print(f"総食品ロス量            : {total_food_loss:.3f} kg (内 加工ロス: {(processing_loss_a + processing_loss_b):.3f} kg)")
    print(f"Middle 環境負荷         : {total_impact:.3f} ポイント")

# =================================================================
# モジュール7: ダッシュボード
# =================================================================
def create_dashboard(df):
    if df.empty: return

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=("在庫レベル推移 (kg)", "加工品生産量 (kg)", "廃棄量の内訳 (kg)", "原料の用途別配分推移 (kg)"),
        specs=[[{}, {}], [{}, {}]]
    )

    inv_post_cols = ['Inv_post_f', 'Inv_post_a', 'Inv_post_b']
    inv_pre_cols = ['Inv_pre_a', 'Inv_pre_b']
    colors = px.colors.qualitative.Plotly

    # 1. 在庫グラフ (左上)
    for i, col in enumerate(inv_post_cols):
        fig.add_trace(go.Bar(x=df['Period'], y=df[col] , name=f'加工後-{col[-1]}', legendgroup='1_inv', marker_color=colors[i]), row=1, col=1)
    for i, col in enumerate(inv_pre_cols):
         fig.add_trace(go.Bar(x=df['Period'], y=df[col], name=f'加工前-{col[-1]}', legendgroup='1_inv', marker_color=colors[i+3], opacity=0.7), row=1, col=1)

    # 2. 生産量グラフ (右上)
    fig.add_trace(go.Bar(x=df['Period'], y=df['P_a'], name='タルト(a)生産', legendgroup='2_prod', marker_color=colors[1]), row=1, col=2)
    fig.add_trace(go.Bar(x=df['Period'], y=df['P_b'], name='シロップ(b)生産', legendgroup='2_prod', marker_color=colors[2]), row=1, col=2)

    # 3. 廃棄量グラフ (左下)
    waste_cols = ['W_post_f', 'W_post_a', 'W_post_b', 'W_pre_a', 'W_pre_b', 'W_proc_a', 'W_proc_b']
    for i, col in enumerate(waste_cols):
        # 該当カラムが存在する場合のみ描画
        if col in df.columns:
            fig.add_trace(go.Bar(x=df['Period'], y=df[col], name=col, legendgroup='3_waste', marker_color=px.colors.qualitative.Pastel[i%10]), row=2, col=1)

    # 4. 原料配分推移グラフ (右下) - 積み上げ棒グラフ
    alloc_cols = ['X_f', 'X_a', 'X_b']
    alloc_names = {'X_f': '配分-生鮮(f)', 'X_a': '配分-タルト用(a)', 'X_b': '配分-シロップ用(b)'}

    for i, col in enumerate(alloc_cols):
        fig.add_trace(go.Bar(
            x=df['Period'], y=df[col],
            name=alloc_names[col],
            legendgroup='4_alloc',
            marker_color=colors[i] # 在庫グラフの色と合わせる
        ), row=2, col=2)

    fig.update_layout(
        height=800,
        title_text="詳細シミュレーションダッシュボード",
        legend_title_text="凡例",
        barmode='stack',
        bargap=0.15
    )

    # 軸ラベルの設定
    fig.update_xaxes(title_text="期間 (Month)", tickmode='linear')
    fig.update_yaxes(title_text="重量 (kg)", row=1, col=1)
    fig.update_yaxes(title_text="重量 (kg)", row=1, col=2)
    fig.update_yaxes(title_text="重量 (kg)", row=2, col=1)
    fig.update_yaxes(title_text="重量 (kg)", row=2, col=2)

    fig.show()

# =================================================================
# モジュール8: メイン実行ブロック トップレベル環境 一番最初に実行される
# =================================================================
if __name__ == '__main__':
    SIMULATION_PERIODS = 12
    PLANNING_HORIZON = 6
    TOTAL_SCENARIO_PERIODS = SIMULATION_PERIODS + PLANNING_HORIZON # 合計18カ月分

    # 需要シナリオ生成
    print("="*80)
    print(f"シナリオ生成中 (期間: {SIMULATION_PERIODS}, Horizon: {PLANNING_HORIZON})...")
    master_scenarios = []
    for t in range(TOTAL_SCENARIO_PERIODS):
        scenarios_for_period_t = {
            f"s_{i+1}": {
                'prob': 1 / num_scenarios_per_period, # そのシナリオが起こる確率
                'd': {p: max(0, np.random.normal(demand_params['mean'][p], demand_params['std'][p])) for p in products} # 需要生成
            } for i in range(num_scenarios_per_period)
        }
        master_scenarios.append(scenarios_for_period_t)

    # 正規化基準値の計算
    try:
        norm_limits = calculate_normalization_bounds( # モジュール3-2（正規化の基準値を探す）
            initial_inventory_post, initial_inventory_pre, C_supply, master_scenarios[0:PLANNING_HORIZON], params, capacities
        )
    except Exception as e:
        print(f"Error during normalization: {e}")
        norm_limits = None

    if norm_limits:
        # --- パレートフロンティア分析 ---
        print("\n" + "="*80)
        print("パレートフロンティア分析を実行中 (λ: 0.0 -> 1.0)")
        print("="*80)

   # グラフ描画に必要な変数の定義
        grid_color_mono = 'darkgray'

        # np.linspace(開始,終了,間を何等分するか)
        env_weights_linear = np.linspace(0.0,1.0,11)
        # モジュール5（パレートフロンティア分析）
        df_results_linear = run_pareto_analysis(env_weights_linear, SIMULATION_PERIODS, master_scenarios, norm_limits, title="詳細スキャン")

        if not df_results_linear.empty:
            print("\n--- λ（環境重視度）と各指標の関係 ---")
            fig_lambda_linear_separate = make_subplots(
                rows=2, cols=1,
                shared_xaxes=True,
                vertical_spacing=0.05,
            )

            # 総コスト
            fig_lambda_linear_separate.add_trace(go.Scatter(
                x=df_results_linear['weight'], y=df_results_linear['total_cost'],
                name='総コスト', mode='lines+markers',
                line=dict( dash='solid', width=2),
                marker=dict(symbol='circle', size=6)
            ), row=1, col=1)

            # 総環境負荷
            fig_lambda_linear_separate.add_trace(go.Scatter(
                x=df_results_linear['weight'], y=df_results_linear['total_env_score'],
                name='総環境負荷', mode='lines+markers',
                line=dict(dash='solid', width=2),
                marker=dict(symbol='circle', size=6)
            ), row=2, col=1)

            fig_lambda_linear_separate.update_layout(
                    title_text=f'λ={zoom_range_start}～{zoom_range_end}における各指標の関係',
                    height=600,
                    showlegend=False,
                    plot_bgcolor='white',
                    paper_bgcolor='white',
                    title_font=dict(size=20),
                    font_color='black'
            )

            # Y軸 (1段目)
            fig_lambda_linear_separate.update_yaxes(
                  title_text="総コスト（C）",
                  title_font=dict(size=18, color='black'),
                  tickfont=dict(size=14, color='black'),
                  showgrid=True,
                  gridcolor=grid_color_mono,
                  zeroline=False,
                  linecolor='black',
                  row=1, col=1
            )
            # Y軸 (2段目)
            fig_lambda_linear_separate.update_yaxes(
                  title_text="総環境負荷スコア（E）",
                  title_font=dict(size=18, color='black'),
                  tickfont=dict(size=14, color='black'),
                  showgrid=True,
                  gridcolor=grid_color_mono,
                  zeroline=False,
                  linecolor='black',
                  row=2, col=1
            )

            # X軸 (共通)
            fig_lambda_linear_separate.update_xaxes(
                  title_text='λ (環境重視度)',
                  title_font=dict(size=18, color='black'),
                  tickfont=dict(size=14, color='black'),
                  showgrid=False,
                  gridcolor=grid_color_mono,
                  linecolor='black',
                  zeroline=False,
                  row=2, col=1 # 最後のグラフにのみX軸タイトルを表示
            )
            fig_lambda_linear_separate.show()


            # --- パレートフロンティア ---
            print("\n--- グラフ２: パレートフロンティア ---")
            fig_pareto_frontier = go.Figure()
            fig_pareto_frontier.add_trace(go.Scatter(
                x=df_results_linear['total_env_score'],
                y=df_results_linear['total_cost'],
                mode='lines+markers',
                text=df_results_linear['weight'].apply(lambda w: f'λ={w:.1f}'), # ホバーテキスト
                hovertemplate='<b>環境負荷</b>: %{x:.2f}<br><b>総コスト</b>: %{y:,.0f} 円<br><b>%{text}</b><extra></extra>',
                line=dict(color='green'),   # 線の色
                marker=dict(color='green')  # 点の色
            ))

            fig_pareto_frontier.update_layout(
                title_text='パレートフロンティア（コストと環境負荷のトレードオフ）',
                xaxis_title='総環境負荷スコア (E)',
                yaxis_title='総コスト (C)',
                height=500,
                showlegend=False,
                title_font=dict(size=20),
                plot_bgcolor='white',     # グラフ背景を白に
                paper_bgcolor='white',    # グラフ全体の背景を白に
                font_color='black',       # フォントを黒に
                # X軸のスタイル
                xaxis=dict(
                    showgrid=True,
                    gridcolor=grid_color_mono,
                    linecolor='black',
                    zeroline=False
                ),
                # Y軸のスタイル
                yaxis=dict(
                    showgrid=True,
                    gridcolor=grid_color_mono,
                    linecolor='black',
                    zeroline=False
                )
            )

            fig_pareto_frontier.show()

            print("\n" + "="*80)
            print("【比較表】")
            print("="*80)
            print(df_results_linear[['weight', 'total_cost', 'total_env_score']].round(2))

        # --- 詳細分析 (λ=0.5) ---
        chosen_weight = 0.5
        print("\n" + "="*80)
        print(f"詳細分析 (λ={chosen_weight})")
        print("="*80)

        # 1期間目モデル詳細
        # モジュール3-1（最適化）λ=0.5の12カ月のみを再度計算する
        _, first_period_model = solve_multi_objective_model(initial_inventory_post, initial_inventory_pre, C_supply, master_scenarios[0:PLANNING_HORIZON], params, capacities, chosen_weight, norm_limits)
        # モジュール6（1期間の詳細）
        if first_period_model: analyze_and_print_period_details(first_period_model, master_scenarios[0], params)

        # 12期間シミュレーション & 論文用テーブル
        # モジュール4（ローリングホライズン)を試行、選択したλで1回のみ
        detailed_df = run_two_stage_simulation(SIMULATION_PERIODS, chosen_weight, initial_inventory_post, initial_inventory_pre, master_scenarios, norm_limits, verbose=False)
        if not detailed_df.empty:
            print("\n" + "="*100)
            print(f"【詳細データ表】 (λ = {chosen_weight}, Environment Factor: Middle)")
            print("="*100)

            # 学術論文テーブル用フォーマット
            display_df = detailed_df[['Period', 'Total Cost', 'Holding Cost', 'Processing Cost', 'Waste Cost', 'Opportunity Cost', 'Total Env Score', 'Waste Env', 'Process Env']].copy()
            display_df.columns = ['期(t)', '総コスト', '保管費用', '加工費用', '廃棄費用', '機会損失', '環境負荷(総)', '環境(廃棄)', '環境(加工)']

            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 1000)
            pd.set_option('display.float_format', '{:,.2f}'.format)

            print(display_df.to_string(index=False))
            print("-" * 100)

            totals = display_df.sum(numeric_only=True)
            totals['期(t)'] = float('nan')
            print(f"合計{'':<4} {totals['総コスト']:,.2f}   {totals['保管費用']:,.2f}     {totals['加工費用']:,.2f}     {totals['廃棄費用']:,.2f}    {totals['機会損失']:,.2f}    {totals['環境負荷(総)']:,.2f}     {totals['環境(廃棄)']:,.2f}     {totals['環境(加工)']:,.2f}")
            print("="*100)

            create_dashboard(detailed_df)

シナリオ生成中 (期間: 12, Horizon: 6)...
正規化のための基準値を計算中 (Horizon: 6 months)...
正規化のための基準値を計算中 (Horizon: 6 months, Grid size: 11)...
  Cost Range (Total for Horizon): 116,784 ~ 185,545
  Env  Range (Total for Horizon): 50.255 ~ 71.555

パレートフロンティア分析を実行中 (λ: 0.0 -> 1.0)
詳細スキャンを開始...

--- λ（環境重視度）と各指標の関係 ---



--- グラフ２: パレートフロンティア ---



【比較表】
    weight  total_cost  total_env_score
0     0.00  227,415.25           140.54
1     0.10  228,203.03           136.43
2     0.20  231,360.62           132.14
3     0.30  235,507.71           128.23
4     0.40  240,328.61           125.53
5     0.50  246,020.26           123.30
6     0.60  250,186.61           122.19
7     0.70  253,878.15           121.28
8     0.80  257,434.21           120.84
9     0.90  261,686.53           120.51
10    1.00  267,249.75           120.22

詳細分析 (λ=0.5)
期待総コスト (当期分): 23,107.411 円

--- 総コストの内訳 (当期分) ---
  期待保管コスト        : 14.802
  加工コスト            : 8,875.416
  期待廃棄コスト        : 6,427.432 (内 加工廃棄: 3,927.432)
  期待機会損失コスト    : 7,789.762
総食品ロス量            : 6.025 kg (内 加工ロス: 3.525 kg)
Middle 環境負荷         : 12.350 ポイント

【詳細データ表】 (λ = 0.5, Environment Factor: Middle)
 期(t)      総コスト  保管費用      加工費用     廃棄費用     機会損失  環境負荷(総)  環境(廃棄)  環境(加工)
    1 23,107.41 14.80  8,875.42 6,427.43 7,789.76    12.35    7.85    4.50
    2 20,589.22 16.27  9,951.59 4,54

In [ ]:
 detailed_df.to_csv('simulation_result.csv')